# 🌾 Grain Recommender Agent — 실행 튜토리얼

이 노트북은 **규칙 기반 그레인 추천 에이전트**의 단일 턴 플로우를
직접 실행하고 결과를 살펴볼 수 있는 가이드입니다. 아래 순서를 따라가면
자유 텍스트 입력 → 정보 추출 → 검증 → 추천 → 설명/포매팅까지 한 번에 수행되는
동작을 확인할 수 있습니다.

## 1. 사전 준비
- 프로젝트 루트(이 노트북이 있는 경로)에서 `python -m venv .venv && source .venv/bin/activate` 등으로 가상환경을 활성화하고,
  `pip install -r requirements.txt`로 의존성을 설치합니다.
- (선택) FastAPI 데모를 실행하려면 `fastapi`, `uvicorn`, `pydantic` 패키지가 추가로 필요합니다.

## 2. LLM 설정 (선택)
규칙 기반 추출만 사용할 경우 이 단계를 건너뛰어도 됩니다. LLM 기반 추출을 사용하려면 아래 환경 변수를 설정하세요.
- OpenAI: `LLM_PROVIDER=openai`, `OPENAI_API_KEY`, 필요 시 `OPENAI_MODEL`
- Gemini: `LLM_PROVIDER=gemini`, `GEMINI_API_KEY`, 필요 시 `GEMINI_MODEL`
아래 셀에 키를 직접 입력하면 노트북 세션 동안만 적용됩니다.


In [ ]:
import os

# 아래 값을 채우면 노트북 세션에서만 일시적으로 환경 변수를 설정합니다.
LLM_PROVIDER = os.getenv("LLM_PROVIDER", "")  # "openai" 또는 "gemini"
OPENAI_API_KEY = ""  # 예: "sk-..."
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
GEMINI_API_KEY = ""  # 예: "your-gemini-key"
GEMINI_MODEL = os.getenv("GEMINI_MODEL", "gemini-1.5-flash")
GRAIN_AGENT_USE_LLM = os.getenv("GRAIN_AGENT_USE_LLM", "")  # "true"로 설정하면 LLM 모드를 사용합니다.

if LLM_PROVIDER:
    os.environ["LLM_PROVIDER"] = LLM_PROVIDER

if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
if OPENAI_MODEL:
    os.environ["OPENAI_MODEL"] = OPENAI_MODEL

if GEMINI_API_KEY:
    os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
if GEMINI_MODEL:
    os.environ["GEMINI_MODEL"] = GEMINI_MODEL

if GRAIN_AGENT_USE_LLM:
    os.environ["GRAIN_AGENT_USE_LLM"] = GRAIN_AGENT_USE_LLM

print("LLM_PROVIDER =", os.getenv("LLM_PROVIDER", ""))
print("OPENAI_API_KEY 설정됨?", bool(os.getenv("OPENAI_API_KEY")))
print("GEMINI_API_KEY 설정됨?", bool(os.getenv("GEMINI_API_KEY")))
print("GRAIN_AGENT_USE_LLM =", os.getenv("GRAIN_AGENT_USE_LLM", ""))


## 3. 프로젝트 경로 확인
아래 셀은 노트북의 현재 작업 경로를 확인하고, `app/` 폴더가 존재하는지 간단히 검사합니다.
필요하다면 `PROJECT_ROOT`를 수정해 주세요.

In [ ]:

from pathlib import Path

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "app").exists():
    raise FileNotFoundError(f"app/ 폴더를 찾을 수 없습니다: {PROJECT_ROOT}")

print(f"PROJECT_ROOT = {PROJECT_ROOT}")


## 4. 모듈 임포트 및 설정
`sys.path`에 프로젝트 루트를 추가한 뒤, 에이전트 플로우와 스키마/룰 설정을 로드합니다.

In [ ]:

import sys

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from pprint import pprint
from app.agent.graph import run_agent_flow, SURVEY_SCHEMA, RULE_WEIGHTS

print("Imports OK. 스키마 필수 필드:", SURVEY_SCHEMA["required"])


## 5. 추천 플로우 실행
대표적인 입력 예시를 사용해 추천 결과를 확인합니다. 메시지와 페이로드(JSON)를 함께 출력하도록 구성했습니다.

In [ ]:

example_text = "혈당 관리가 필요하고 주 6번 정도 밥을 먹어요. 찰진 식감이 좋고 보리는 빼주세요. 글루텐은 피하고 싶어요."

result = run_agent_flow(example_text, user_id="demo-user")
print(result["message"])
print("\nPayload:")
pprint(result["payload"])


## 6. 검증 실패/재질문 흐름 확인
필수 정보가 부족한 입력을 전달하면 검증 도구가 재질문 메시지를 반환합니다.

In [ ]:

missing_text = "다이어트 목적이에요."

reask = run_agent_flow(missing_text, user_id="demo-user")
print(reask["message"])
print("\nPayload:")
pprint(reask["payload"])


## 7. 스키마와 규칙 살펴보기
추출/검증/추천에 사용되는 스키마와 규칙 설정을 직접 확인할 수 있습니다.
필요시 이 값을 편집해 자신만의 규칙을 정의할 수 있습니다.

In [ ]:

print("SURVEY_SCHEMA:")
pprint(SURVEY_SCHEMA)

print("\nRULE_WEIGHTS (일부):")
for key, value in list(RULE_WEIGHTS.items())[:5]:
    print(f"- {key}: {value}")


## 8. 자유 입력 실험용 헬퍼
`try_flow` 함수에 임의의 문장을 넣어 결과를 반복적으로 확인할 수 있습니다.

In [ ]:

def try_flow(text: str, user_id: str = "demo-user"):
    """Utility helper to run the agent flow and pretty-print the results."""
    output = run_agent_flow(text, user_id=user_id)
    print("\n=== 메시지 ===")
    print(output["message"])
    print("\n=== 페이로드 ===")
    pprint(output["payload"])
    return output

# 사용 예시
_ = try_flow("근력 강화 목표고 주 4번 밥을 먹어요. 고슬한 식감을 원하고 보리는 피하고 싶어요.")


---

필요하다면 FastAPI 엔드포인트(`app/api/main.py`)를 이용해 REST 호출을 테스트할 수도 있습니다.
이 경우 `uvicorn`과 `fastapi` 패키지를 설치한 뒤 별도 터미널에서 `uvicorn app.api.main:app --reload`
명령을 실행하고, `requests` 혹은 브라우저로 `/recommend`를 호출해 보세요.